# 🧠 EEG-to-Text Seq2Seq Training (BEST APPROACH)

## 📋 Overview
- **Model**: Sequence-to-Sequence LSTM with Attention
- **Approach**: Generates sentences word-by-word (like machine translation)
- **Expected Results**: MUCH BETTER than classification
- **Training Time**: 3-5 hours on CPU

## ✅ Why Seq2Seq is Better
1. **Partial credit**: Gets credit for predicting some words correctly
2. **Works with 95 classes**: Can learn from limited data per class
3. **More realistic**: Like translating EEG → English text
4. **Better metrics**: Word Error Rate (WER), not just exact match
5. **Flexible**: Can potentially generate new sentences

## 🎯 Expected Results (95 classes)
- **Word Error Rate (WER)**: 40-60% (lower is better)
- **Exact Match Accuracy**: 10-20% (higher than classification)
- **Partial correctness**: Gets 60-80% of words right even if sentence is wrong

---

## Step 1: Mount Google Drive

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
print("\n✅ Google Drive mounted successfully!")

## Step 2: Clone Repository from GitHub

In [ ]:
import os

# Clone repository (or pull latest changes if already exists)
if not os.path.exists('/content/ML-Project-Data'):
    print("📥 Cloning repository from GitHub...")
    !git clone https://github.com/Tejas-Chakkarwar/ML-Project-Data.git
    print("✅ Repository cloned!")
else:
    print("✅ Repository already exists")
    print("📥 Pulling latest changes...")
    !cd /content/ML-Project-Data && git pull origin main

# Change to project directory
os.chdir('/content/ML-Project-Data')
print(f"\n✅ Working directory: {os.getcwd()}")

# Verify Seq2Seq files
print("\n📋 Verifying Seq2Seq files:")
print(f"  lstm_approach/: {'✅' if os.path.exists('lstm_approach') else '❌ MISSING'}")
print(f"  lstm_approach/train_seq2seq.py: {'✅' if os.path.exists('lstm_approach/train_seq2seq.py') else '❌ MISSING'}")
print(f"  lstm_approach/seq2seq_model.py: {'✅' if os.path.exists('lstm_approach/seq2seq_model.py') else '❌ MISSING'}")
print(f"  lstm_approach/vocabulary.py: {'✅' if os.path.exists('lstm_approach/vocabulary.py') else '❌ MISSING'}")

## Step 3: Install Dependencies

In [ ]:
# Install required packages
!pip install -q torch numpy pandas scikit-learn tqdm

# Check GPU availability
import torch

print("\n📊 System Information:")
print(f"  GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"  GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"  GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print("\n  ✅ GPU enabled - Training will be much faster!")
    DEVICE = 'cuda'
else:
    print("\n  ⚠️ No GPU detected - using CPU")
    print("  Training will take 3-5 hours")
    DEVICE = 'cpu'

print("\n✅ Dependencies installed!")

## Step 4: Copy Dataset from Google Drive

⚠️ **IMPORTANT**: Update the `SOURCE` path below to match your dataset location!

Common paths:
- `/content/drive/MyDrive/ML_Project_Dataset`
- `/content/drive/MyDrive/Colab Notebooks/dataset`

In [ ]:
import time
import shutil
import os
import glob

print("=" * 70)
print("COPYING DATASET FROM GOOGLE DRIVE")
print("=" * 70)

# ⚠️ UPDATE THIS PATH TO YOUR ACTUAL DATASET LOCATION!
SOURCE = '/content/drive/MyDrive/ML_Project_Dataset'  # ← Change this!
DEST = '/content/ML-Project-Data/processed_data'

# Verify source exists
if not os.path.exists(SOURCE):
    print(f"\n❌ ERROR: Source path not found!")
    print(f"   Path: {SOURCE}")
    print("\n📁 Available folders in MyDrive:")
    for item in os.listdir('/content/drive/MyDrive'):
        item_path = os.path.join('/content/drive/MyDrive', item)
        if os.path.isdir(item_path):
            print(f"   📁 {item}/")
    print("\n⚠️ Please update the SOURCE path in the cell above!")
else:
    # Create destination
    os.makedirs(DEST, exist_ok=True)
    
    # Get list of files
    all_files = sorted(os.listdir(SOURCE))
    total_files = len(all_files)
    
    print(f"\n✅ Source found: {SOURCE}")
    print(f"📊 Total files to copy: {total_files:,}")
    
    # Check what's already copied
    already_copied = set(os.listdir(DEST)) if os.path.exists(DEST) else set()
    print(f"📊 Already copied: {len(already_copied):,}")
    print(f"📊 Remaining: {total_files - len(already_copied):,}\n")
    
    # Copy files with progress
    copied = 0
    failed = []
    start_time = time.time()
    
    for i, filename in enumerate(all_files, 1):
        if filename in already_copied:
            continue
        
        src_path = os.path.join(SOURCE, filename)
        dst_path = os.path.join(DEST, filename)
        
        try:
            if os.path.isfile(src_path):
                shutil.copy2(src_path, dst_path)
                copied += 1
        except Exception as e:
            failed.append(filename)
        
        if (i % 1000 == 0) or (i == total_files):
            elapsed = time.time() - start_time
            print(f"   [{i:,}/{total_files:,}] Progress ({elapsed/60:.1f} min)")
    
    elapsed = time.time() - start_time
    
    print("\n" + "=" * 70)
    print("COPY COMPLETE")
    print("=" * 70)
    print(f"⏱️  Time: {elapsed/60:.1f} minutes")
    print(f"✅ Copied: {copied:,} files")
    print(f"📊 Already existed: {len(already_copied):,} files")
    print(f"❌ Failed: {len(failed)} files")
    
    # Verify
    csv_files = glob.glob(f'{DEST}/rawdata_*.csv')
    mapping_exists = os.path.exists(f'{DEST}/sentence_mapping.csv')
    
    print(f"\n📋 Final Verification:")
    print(f"   CSV files: {len(csv_files):,}")
    print(f"   Mapping file: {'✅' if mapping_exists else '❌'}")
    
    if len(csv_files) >= 5900 and mapping_exists:
        print("\n🎉 SUCCESS! Data is ready for training!")
    else:
        print(f"\n⚠️  Warning: Only {len(csv_files):,} files (expected ~5,915)")
    
    print("=" * 70)

## Step 5: Train Seq2Seq Model (95 Classes)

### 🎯 Configuration:
- **Classes**: ~95 (min_samples=18)
- **Approach**: Generate sentences word-by-word
- **Vocabulary**: Built from all training sentences
- **Augmentation**: 6x (more training data)

### ⏱️ Expected:
- **Training time**: 3-5 hours on CPU / 1-2 hours on GPU
- **Word Error Rate**: 40-60% (lower is better)
- **Exact Match**: 10-20% (better than classification's 4-7%)
- **Partial correctness**: 60-80% of words correct

---

**⚠️ This cell will take 3-5 hours to complete. Don't close your browser!**

In [ ]:
import os
import time

os.chdir('/content/ML-Project-Data/lstm_approach')

print("=" * 70)
print("🚀 SEQ2SEQ EEG-TO-TEXT TRAINING")
print("=" * 70)
print("\n🎯 Configuration:")
print("  ✅ Classes: ~95 (min_samples=18)")
print("  ✅ Approach: Word-by-word generation")
print("  ✅ Augmentation: 6x")
print("  ✅ Model: Encoder-Decoder LSTM with Attention")
print(f"  ✅ Device: {DEVICE}")
print()
print("💾 Expected Memory: ~6-8 GB RAM")
print("⏱️  Expected Time: 3-5 hours (CPU) / 1-2 hours (GPU)")
print("🎯 Target WER: 40-60%")
print("=" * 70 + "\n")

start = time.time()

!python train_seq2seq.py \
  --min-samples 18 \
  --num-aug 6 \
  --batch-size 16 \
  --epochs 40 \
  --lr 0.001 \
  --teacher-forcing 0.5 \
  --device {DEVICE} \
  --max-len 60

elapsed = time.time() - start

print("\n" + "=" * 70)
print(f"🎉 TRAINING COMPLETED IN {elapsed/60:.1f} MINUTES ({elapsed/3600:.1f} HOURS)")
print("=" * 70)

## Step 6: Save Models to Google Drive

In [ ]:
import os
import shutil

# Create destination in Google Drive
DRIVE_MODELS_DIR = '/content/drive/MyDrive/ML_Project_Seq2Seq_Models'
os.makedirs(DRIVE_MODELS_DIR, exist_ok=True)

# Copy checkpoints
LOCAL_CHECKPOINT_DIR = '/content/ML-Project-Data/checkpoints'

if os.path.exists(LOCAL_CHECKPOINT_DIR):
    print("📦 Saving Seq2Seq model to Google Drive...\n")
    
    for filename in os.listdir(LOCAL_CHECKPOINT_DIR):
        if filename.endswith('.pth'):
            src = os.path.join(LOCAL_CHECKPOINT_DIR, filename)
            dst = os.path.join(DRIVE_MODELS_DIR, filename)
            
            shutil.copy2(src, dst)
            size_mb = os.path.getsize(dst) / 1e6
            print(f"✅ {filename} ({size_mb:.1f} MB)")
    
    print(f"\n✅ Models saved to: {DRIVE_MODELS_DIR}")
    print("   These will persist even after session ends!")
else:
    print("⚠️  No checkpoints found. Did training complete successfully?")

## Step 7 (Optional): Quick Test on 5 Classes

If you want faster results for demonstration, train on 5 most common sentences.
Expected: **60-80% exact match**, **20-30% WER**

In [ ]:
import os
import time

os.chdir('/content/ML-Project-Data/lstm_approach')

print("=" * 70)
print("🚀 SEQ2SEQ TRAINING - 5 CLASSES (DEMO)")
print("=" * 70)

start = time.time()

!python train_seq2seq.py \
  --min-samples 20 \
  --num-aug 6 \
  --batch-size 16 \
  --epochs 30 \
  --lr 0.001 \
  --teacher-forcing 0.5 \
  --device {DEVICE} \
  --max-len 40

elapsed = time.time() - start

print("\n" + "=" * 70)
print(f"🎉 TRAINING COMPLETED IN {elapsed/60:.1f} MINUTES")
print("=" * 70)

---

## 📊 Understanding Your Results

### Metrics Explained:

1. **Exact Match Accuracy**: Percentage of sentences predicted perfectly
   - 95 classes: 10-20% is GOOD (much better than 4-7% classification)
   - 5 classes: 60-80% is EXCELLENT

2. **Word Error Rate (WER)**: Percentage of words that are wrong
   - Lower is better
   - 40-60% means 40-60% of words are wrong (or 40-60% correct)
   - This shows partial understanding

### Why Seq2Seq is Better:
- Classification: **All or nothing** (entire sentence must be correct)
- Seq2Seq: **Partial credit** (gets points for correct words)

### Example:
```
True: "The cat sat on the mat"
Pred: "The dog sat on the chair"

Classification: 0% (wrong sentence)
Seq2Seq: 66% (4/6 words correct, WER = 33%)
```

---

## 🎯 For Your Presentation

**Recommended approach:**
1. Show 5-class results (60-80% exact match)
2. Explain Seq2Seq generates word-by-word
3. Show examples of partial correctness
4. Compare with classification (4-7% for 95 classes)
5. Mention 95-class WER shows model understands patterns

---